In [ ]:
import pandas as pd
import numpy as np

# Define the path to your dataset
file_path = '/content/Crop_recommendation.csv'

# Load the dataset
data = pd.read_csv(file_path)

# Exclude the first and last columns
data = data.iloc[:, :-1]

# Step 1: Get user input for criteria selection
def get_criteria_selection(data):
    available_columns = data.columns  # Use remaining columns
    print(f"Available Criteria: {list(available_columns)}")

    num_criteria = int(input("Enter how many criteria you want to select: "))

    selected_criteria = []
    for i in range(num_criteria):
        criterion = input(f"Select criterion {i+1}: ")
        if criterion in available_columns:
            selected_criteria.append(criterion)
        else:
            print(f"Invalid criterion. Please select from available criteria: {available_columns}")

    print(f"Selected Criteria: {selected_criteria}")
    return selected_criteria

# Step 2: Get user-provided values for selected criteria
def get_user_values(selected_criteria):
    user_values = {}
    for criterion in selected_criteria:
        value = float(input(f"Enter value for {criterion}: "))
        user_values[criterion] = value
    return user_values

# Step 3: Perform correlation analysis
def compute_correlation(data, selected_criteria):
    correlation_matrix = data[selected_criteria].corr()
    print("\nCorrelation Matrix:")
    print(correlation_matrix.round(4))
    return correlation_matrix

# Step 4: Map Correlation Values to Saaty Scale
def map_to_saaty_scale(corr_value):
    if corr_value > 0.9:
        return 9
    elif corr_value > 0.7:
        return 7
    elif corr_value > 0.5:
        return 5
    elif corr_value > 0.3:
        return 3
    elif corr_value > 0.1:
        return 2
    elif corr_value >= -0.1:
        return 1
    elif corr_value > -0.3:
        return 1/2
    elif corr_value > -0.5:
        return 1/3
    elif corr_value > -0.7:
        return 1/5
    elif corr_value > -0.9:
        return 1/7
    else:
        return 1/9

# Step 5: Construct Pairwise Comparison Matrix
def construct_pairwise_matrix(correlation_matrix):
    criteria = correlation_matrix.columns
    num_criteria = len(criteria)
    pairwise_matrix = np.ones((num_criteria, num_criteria))

    # Saaty scale mapping based on correlation
    for i in range(num_criteria):
        for j in range(i + 1, num_criteria):
            corr_value = correlation_matrix.iloc[i, j]
            pairwise_value = map_to_saaty_scale(corr_value)
            pairwise_matrix[i, j] = pairwise_value
            pairwise_matrix[j, i] = 1 / pairwise_value  # Inverse for reverse comparison

    # Convert to DataFrame for readability
    pairwise_df = pd.DataFrame(pairwise_matrix, index=criteria, columns=criteria)
    print("\nPairwise Comparison Matrix:")
    print(pairwise_df.round(4))
    return pairwise_df

# Run the process
# Get user input for criteria selection
selected_criteria = get_criteria_selection(data)

# Get user-provided values for each selected criterion
user_values = get_user_values(selected_criteria)

# Perform correlation analysis between criteria
correlation_matrix = compute_correlation(data, selected_criteria)

# Construct pairwise comparison matrix based on correlation
pairwise_matrix = construct_pairwise_matrix(correlation_matrix)

Available Criteria: ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']
Enter how many criteria you want to select: 3
Select criterion 1: N
Select criterion 2: P
Select criterion 3: K
Selected Criteria: ['N', 'P', 'K']
Enter value for N: 85
Enter value for P: 58
Enter value for K: 41

Correlation Matrix:
        N       P       K
N  1.0000 -0.2315 -0.1405
P -0.2315  1.0000  0.7362
K -0.1405  0.7362  1.0000

Pairwise Comparison Matrix:
     N       P    K
N  1.0  0.5000  0.5
P  2.0  1.0000  7.0
K  2.0  0.1429  1.0


In [ ]:
import numpy as np

# Function to normalize the matrix
def normalize_matrix(matrix):
    matrix = np.array(matrix, dtype=float)
    col_sums = np.sum(matrix, axis=0)
    normalized_matrix = matrix / col_sums
    return normalized_matrix

# Function to compute weights manually
def compute_weights(normalized_matrix):
    # Sum each row of the normalized matrix
    row_sums = np.sum(normalized_matrix, axis=1)
    n = len(row_sums)
    weights = row_sums / np.sum(row_sums)  # Normalize to make sum to 1
    return weights

# Function to compute CI and CR
def compute_consistency_measures(matrix, max_eigenvalue, ri):
    n = matrix.shape[0]
    ci = (max_eigenvalue - n) / (n - 1)
    cr = ci / ri
    return ci, cr

# Random Index for n = 3 (from Saaty's table)
RI = 1.41

# Round the pairwise matrix
rounded_pairwise_matrix = np.round(pairwise_matrix, 4)

# Normalize the pairwise comparison matrix
normalized_matrix = normalize_matrix(rounded_pairwise_matrix)

# Round the normalized matrix
rounded_normalized_matrix = np.round(normalized_matrix, 4)

# Compute weights manually
weights = compute_weights(rounded_normalized_matrix)

# Round the weights
rounded_weights = np.round(weights, 4)

# Compute eigenvalues and eigenvectors
eigenvalues = np.real(np.linalg.eigvals(rounded_pairwise_matrix))
max_eigenvalue = np.real(np.max(eigenvalues))

# Round eigenvalues and max eigenvalue
rounded_eigenvalues = np.round(eigenvalues, 4)
rounded_max_eigenvalue = np.round(max_eigenvalue, 4)

# Compute consistency measures
ci, cr = compute_consistency_measures(rounded_pairwise_matrix, rounded_max_eigenvalue, RI)

# Round CI and CR
rounded_ci = np.round(ci, 4)
rounded_cr = np.round(cr, 4)

# Print results
print("\nPairwise Comparison Matrix:")
print(rounded_pairwise_matrix)
print("\nNormalized Matrix:")
print(rounded_normalized_matrix)
print("\nWeights:")
print(rounded_weights)
print("\nEigenvalues:")
print(rounded_eigenvalues)
print("\nMax Eigenvalue:")
print(rounded_max_eigenvalue)
print("\nConsistency Measures:")
print(f"Consistency Index (CI): {rounded_ci}")
print(f"Consistency Ratio (CR): {rounded_cr}")

if rounded_cr < 0.1:
    print("The pairwise comparison matrix is consistent.")
else:
    print("The pairwise comparison matrix is not consistent. Consider revising your comparisons.")


Pairwise Comparison Matrix:
     N       P    K
N  1.0  0.5000  0.5
P  2.0  1.0000  7.0
K  2.0  0.1429  1.0

Normalized Matrix:
[[0.2    0.3043 0.0588]
 [0.4    0.6087 0.8235]
 [0.4    0.087  0.1176]]

Weights:
[0.1877 0.6108 0.2015]

Eigenvalues:
[ 3.4357 -0.2179 -0.2179]

Max Eigenvalue:
3.4357

Consistency Measures:
Consistency Index (CI): 0.2179
Consistency Ratio (CR): 0.1545
The pairwise comparison matrix is not consistent. Consider revising your comparisons.


In [ ]:
import pandas as pd

# Function to check if all selected criteria values are together in any row
def check_criteria_validity(data, selected_criteria_values):
    # Round the criteria values to the nearest whole number
    rounded_selected_values = {criterion: round(value)
                               for criterion, value in selected_criteria_values.items()}

    # Round the values in the dataset to the nearest whole number
    rounded_data = data.copy()
    for column in selected_criteria_values.keys():
        if column in rounded_data.columns:
            rounded_data[column] = rounded_data[column].round()

    # Check if all selected criteria values are present in any row
    for index, row in rounded_data.iterrows():
        # Create a set of values from the row that match the selected criteria columns
        row_values = {column: row[column] for column in selected_criteria_values.keys() if column in row}
        # Check if all selected criteria values are in this row
        if all(rounded_selected_values[criterion] == row_values.get(criterion, None)
               for criterion in selected_criteria_values):
            return True

    return False

# Example user-provided criteria values (replace with actual input)
selected_criteria_values = {
    'N': 35,
    'P': 64,
    'K':78,
    'ph': 7,
}

# Check if the provided criteria values are valid
is_valid = check_criteria_validity(data, selected_criteria_values)

if is_valid:
    print("\nAll provided criteria values are present together in the dataset.")
else:
    print("\nThe provided criteria values are not all present together in any row of the dataset. Check your values or adjust your criteria.")


All provided criteria values are present together in the dataset.


In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = 'Crop_recommendation.csv'
data = pd.read_csv(file_path)

# Exclude the last column (crops) from the data
criteria_data = data.iloc[:, :-1]

# Step 1: Get user input for criteria selection
def get_criteria_selection(data):
    available_columns = data.columns
    print(f"Available Criteria: {list(available_columns)}")

    num_criteria = int(input("Enter how many criteria you want to select: "))
    selected_criteria = []

    for i in range(num_criteria):
        criterion = input(f"Select criterion {i+1}: ")
        if criterion in available_columns:
            selected_criteria.append(criterion)
        else:
            print(f"Invalid criterion. Please select from available criteria: {list(available_columns)}")

    print(f"Selected Criteria: {selected_criteria}")
    return selected_criteria

# Step 2: Get user-provided values for selected criteria
def get_user_values(selected_criteria):
    user_values = {}
    for criterion in selected_criteria:
        value = float(input(f"Enter value for {criterion}: "))
        user_values[criterion] = value
    return user_values

# Step 3: Perform correlation analysis
def compute_correlation(data, selected_criteria):
    correlation_matrix = data[selected_criteria].corr()
    print("\nCorrelation Matrix:")
    print(correlation_matrix.round(4))
    return correlation_matrix

# Step 4: Map correlation values to Saaty's scale
def map_to_saaty_scale(corr_value):
    if corr_value > 0.9:
        return 9
    elif corr_value > 0.7:
        return 7
    elif corr_value > 0.5:
        return 5
    elif corr_value > 0.3:
        return 3
    elif corr_value > 0.1:
        return 2
    elif corr_value >= -0.1:
        return 1
    elif corr_value > -0.3:
        return 1/2
    elif corr_value > -0.5:
        return 1/3
    elif corr_value > -0.7:
        return 1/5
    elif corr_value > -0.9:
        return 1/7
    else:
        return 1/9

# Step 5: Construct pairwise comparison matrix based on correlation values
def construct_pairwise_matrix(correlation_matrix):
    criteria = correlation_matrix.columns
    num_criteria = len(criteria)
    pairwise_matrix = np.ones((num_criteria, num_criteria))

    for i in range(num_criteria):
        for j in range(i + 1, num_criteria):
            corr_value = correlation_matrix.iloc[i, j]
            pairwise_value = map_to_saaty_scale(corr_value)
            pairwise_matrix[i, j] = pairwise_value
            pairwise_matrix[j, i] = 1 / pairwise_value

    pairwise_df = pd.DataFrame(pairwise_matrix, index=criteria, columns=criteria)
    print("\nPairwise Comparison Matrix:")
    print(pairwise_df.round(4))
    return pairwise_df

# Function to normalize the matrix
def normalize_matrix(matrix):
    col_sums = np.sum(matrix, axis=0)
    normalized_matrix = matrix / col_sums
    return normalized_matrix

# Function to compute weights
def compute_weights(normalized_matrix):
    row_sums = np.sum(normalized_matrix, axis=1)
    weights = row_sums / np.sum(row_sums)
    return weights

# Step 6: Automatically calculate local and global weights based on correlation
def calculate_weights(pairwise_matrix):
    normalized_matrix = normalize_matrix(pairwise_matrix)
    weights = compute_weights(normalized_matrix)

    print("\nWeights based on correlation values:")
    print(np.round(weights, 4))

    return weights

# Step 7: Check if all selected criteria values are together in any row
def check_criteria_validity(data, selected_criteria_values):
    rounded_selected_values = {criterion: round(value) for criterion, value in selected_criteria_values.items()}
    rounded_data = data.copy()

    for column in selected_criteria_values.keys():
        if column in rounded_data.columns:
            rounded_data[column] = rounded_data[column].round()

    for index, row in rounded_data.iterrows():
        row_values = {column: row[column] for column in selected_criteria_values.keys() if column in row}
        if all(rounded_selected_values[criterion] == row_values.get(criterion, None)
               for criterion in selected_criteria_values):
            return True

    return False

# Step 8: Rank crops based on criteria and correlation-derived weights
def rank_crops(data, selected_criteria, weights, user_values):
    data['score'] = 0  # Initialize score column
    for i, criterion in enumerate(selected_criteria):
        criterion_index = selected_criteria.index(criterion)
        data['score'] += (data[criterion] - user_values[criterion]) ** 2 * weights[criterion_index]

    best_crop = data.iloc[data['score'].idxmin()]['label']
    print(f"\nRecommended Crop: {best_crop}")

# Main process
selected_criteria = get_criteria_selection(criteria_data)
user_values = get_user_values(selected_criteria)
correlation_matrix = compute_correlation(criteria_data, selected_criteria)
pairwise_matrix = construct_pairwise_matrix(correlation_matrix)
weights = calculate_weights(pairwise_matrix)

# Check if user-provided values are valid in the dataset
is_valid = check_criteria_validity(data, user_values)
if is_valid:
    print("\nAll provided criteria values are present together in the dataset.")
    rank_crops(data, selected_criteria, weights, user_values)
else:
    print("\nThe provided criteria values are not all present together in any row of the dataset. Check your values or adjust your criteria.")

Available Criteria: ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']
Enter how many criteria you want to select: 2
Select criterion 1: N
Select criterion 2: P
Selected Criteria: ['N', 'P']
Enter value for N: 85
Enter value for P: 58

Correlation Matrix:
        N       P
N  1.0000 -0.2315
P -0.2315  1.0000

Pairwise Comparison Matrix:
     N    P
N  1.0  0.5
P  2.0  1.0

Weights based on correlation values:
N    0.3333
P    0.6667
dtype: float64

All provided criteria values are present together in the dataset.

Recommended Crop: rice


<ipython-input-1-17279d8ab162>:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data['score'] += (data[criterion] - user_values[criterion]) ** 2 * weights[criterion_index]


In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = 'Crop_recommendation.csv'
data = pd.read_csv(file_path)

# Function to get user input for criteria selection
def get_criteria_selection(data):
    available_columns = data.columns
    print(f"Available Criteria: {list(available_columns)}")

    num_criteria = int(input("Enter how many criteria you want to select: "))
    selected_criteria = []

    for i in range(num_criteria):
        criterion = input(f"Select criterion {i+1}: ")
        if criterion in available_columns:
            selected_criteria.append(criterion)
        else:
            print(f"Invalid criterion. Please select from available criteria: {list(available_columns)}")

    print(f"Selected Criteria: {selected_criteria}")
    return selected_criteria

# Function to get user-provided values for selected criteria
def get_user_values(selected_criteria):
    user_values = {}
    for criterion in selected_criteria:
        value = float(input(f"Enter value for {criterion}: "))
        user_values[criterion] = value
    return user_values

# Function to compute correlation matrix
def compute_correlation(data, selected_criteria):
    correlation_matrix = data[selected_criteria].corr()
    return correlation_matrix

# Function to calculate weights based on correlation values (AHP)
def calculate_weights(correlation_matrix):
    num_criteria = correlation_matrix.shape[0]
    pairwise_matrix = np.ones((num_criteria, num_criteria))

    for i in range(num_criteria):
        for j in range(i + 1, num_criteria):
            corr_value = correlation_matrix.iloc[i, j]
            pairwise_value = map_to_saaty_scale(corr_value)
            pairwise_matrix[i, j] = pairwise_value
            pairwise_matrix[j, i] = 1 / pairwise_value

    # Normalize the pairwise matrix
    normalized_matrix = pairwise_matrix / np.sum(pairwise_matrix, axis=0)

    # Calculate weights
    weights = np.sum(normalized_matrix, axis=1) / np.sum(normalized_matrix)

    return weights

# Function to map correlation values to Saaty's scale
def map_to_saaty_scale(corr_value):
    if corr_value > 0.9:
        return 9
    elif corr_value > 0.7:
        return 7
    elif corr_value > 0.5:
        return 5
    elif corr_value > 0.3:
        return 3
    elif corr_value > 0.1:
        return 2
    elif corr_value >= -0.1:
        return 1
    elif corr_value > -0.3:
        return 1/2
    elif corr_value > -0.5:
        return 1/3
    elif corr_value > -0.7:
        return 1/5
    elif corr_value > -0.9:
        return 1/7
    else:
        return 1/9

# Function to rank crops based on criteria and weights
def rank_crops(data, selected_criteria, weights, user_values):
    data['score'] = 0  # Initialize score column using .loc to avoid warnings

    # Calculate scores based on user values and weights
    for i, criterion in enumerate(selected_criteria):
        data.loc[:, 'score'] += (data[criterion] - user_values[criterion]) ** 2 * weights[i]

    best_crop_index = data['score'].idxmin()  # Get index of crop with minimum score
    best_crop = data.loc[best_crop_index, 'label']  # Retrieve the crop label

    return best_crop

# Function to evaluate accuracy of recommendations against actual crops in the dataset (for simulation)
def evaluate_accuracy(data, selected_criteria):
    correct_recommendations = 0

    # Loop through all rows in the dataset to simulate user input and evaluate recommendations
    for index in range(len(data)):
        row = data.iloc[index]
        user_values = {criterion: row[criterion] for criterion in selected_criteria}

        recommended_crop = rank_crops(data, selected_criteria,
                                       calculate_weights(compute_correlation(data, selected_criteria)),
                                       user_values)

        if recommended_crop == row['label']:  # Check if recommended crop matches actual label
            correct_recommendations += 1

    # Calculate accuracy as a metric of performance
    total_recommendations = len(data)

    accuracy = (correct_recommendations / total_recommendations) * 100 if total_recommendations > 0 else 0

    return recommended_crop, accuracy

# Main process execution
selected_criteria = get_criteria_selection(data)
user_values = get_user_values(selected_criteria)
correlation_matrix = compute_correlation(data, selected_criteria)
weights = calculate_weights(correlation_matrix)

# Rank crops based on criteria and calculated weights for initial recommendation
recommended_crop = rank_crops(data, selected_criteria, weights, user_values)

# Evaluate recommendations against the entire dataset (simulation) and get accuracy
final_recommended_crop, accuracy = evaluate_accuracy(data, selected_criteria)

# Print only the recommended crop and accuracy without excessive output.
print(f"\nRecommended Crop: {recommended_crop}")
print(f"Accuracy of Recommendations: {accuracy:.2f}%")

Available Criteria: ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall', 'label']
Enter how many criteria you want to select: 3
Select criterion 1: N
Select criterion 2: P
Select criterion 3: K
Selected Criteria: ['N', 'P', 'K']
Enter value for N: 85
Enter value for P: 58
Enter value for K: 41

Recommended Crop: rice
Accuracy of Recommendations: 99.68%
